## Table of Contents

1. [Problem Statement](#section1)<br>
2. [Data Loading and Description](#section2)
3. [Naive Submission 1](#section3)
4. [Machine learning Pipeline](#section4)
5. [Naive Submission 2](#section5)
6. [EDA Preprocessing](#section6)
7. [Model Training](#section7)
5. [Model evaluation](#section5)<br>
    - 5.1 [Model Evaluation using accuracy score](#section501)<br>
    - 5.2 [Model Evaluation using confusion matrix](#section502)<br>
6. [Decision Tree with Gridsearch](#section6)<br>

In [6]:

!unzip /content/drive/MyDrive/Mentoring/BDAIO/Session_DU/titanic.zip -d /content/datasets


Archive:  /content/drive/MyDrive/Mentoring/BDAIO/Session_DU/titanic.zip
  inflating: /content/datasets/gender_submission.csv  
  inflating: /content/datasets/test.csv  
  inflating: /content/datasets/train.csv  


In [7]:

import pandas as pd

train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')